# Projet maison n° 4

In [3]:
# imports
import requests
from bs4 import BeautifulSoup
import re

**Partie A**

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

```python
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"
```

Avec `URL_PAGE2`, la fonction doit retourner :

```json
{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
```

In [4]:
# partie A
#collecte des infos

URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    prices = {}
    res = requests.get(url)
    soup = BeautifulSoup(res.content)
    storage = []
    databases = []
    
    #Extraction des titres
    titre = soup.findAll('h2')
    
    #Extraction du prix
    price = soup.findAll("span", attrs={'class': 'pricing-table-price'})
    
    #Extraction storage et databases
    for li_tag in soup.findAll('li'):
        storage.append(re.findall("(.*GB|.*TB)", li_tag.text))
        databases.append(re.findall("(.*) database", li_tag.text))
    storage = list(filter(None, storage))
    databases = list(filter(None, databases))
    
    #Remplissage du dictionnaire
    prices = {titre[i].text : {'price' : re.findall('\$[0-9]+', price[i].text)[0],
                            'storage' : storage[i][0],
                            'databases' : int(databases[i][0])} for i in range(len(titre))}
    
    
    return prices

get_prices_from_url(URL_PAGE2)
get_prices_from_url(URL_PAGE3)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25},
 'Privilege': {'price': '$99', 'storage': '1TB', 'databases': 100}}

**Partie B**

*L'abus d'alcool est dangereux pour la santé, à consommer avec modération.*

1) Ecrire une fonction `extract_beer_infos()` qui extrait des informations sur une bière du site de bières *beowulf*.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/cuvee-des-trolls.33 

La fonction doit retourner :
```json
{'Nom': 'Cuvée des Trolls',
 'Style': 'Bière Blonde',
 'Contenu': 25,
 'Degré d’alcool': 7.0,
 'Origine': 'Belgique',
 'Brasseur': 'Brasserie Dubuisson Freres'}
```

2) L'URL ci-après retourne un JSON avec une liste de bières :

```python
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"
```

Ecrire une fonction `extract_beer_list_infos(url)` qui prend en argument cet URL et retourne les informations sur une liste de bières du site *beowulf*.

Cette fonction doit retourner la liste des informations obtenues par la fonction précédemment définie `extract_beer_infos()`.

Exemple de retour :

```json
[{'Nom':'Desperados','Style':'Lager','Contenu':33,'Degré d’alcool':5.9,'Origine':'France','Brasseur':'Desperados'},
{'Nom':'La Lager Sans Gluten de Vézelay','Style':'Lager','Contenu':25,'Degré d’alcool':4.0,'Origine':'France','Brasseur':'Brasserie de Vézelay'},
{'Nom':'Mélusine Bio','Style':'Pale Ale','Contenu': 33,'Degré d’alcool': 5.0,'Origine':'France','Brasseur':'Mélusine'},
{'Nom':'La Parisienne Le Titi Parisien','Style':'IPA','Contenu':33,'Degré d’alcool':5.5,'Origine':'France','Brasseur': 'Brasserie la Parisienne'},
{'Nom':'Brasserie De Sutter Brin de Folie','Style':'Bière Blonde','Contenu': 33,'Degré d’alcool':6.5,'Origine':'France','Brasseur':'Brasserie de Sutter'}]
```

**Facultatif**

Chercher comment optimiser cette fonction en utilisant `multiprocessing.Pool()` pour paralléliser les accès web.

In [5]:
# partie B-1

URL_PAGE4 = 'https://www.beerwulf.com/fr-fr/p/bieres/newcastle-brown-ale-fut-2l'

def extract_beer_infos(url):
    
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    div_tag = soup.findAll('div', attrs={'class': 'product-info beer-product-info'})
    name = soup.find('h1', attrs={'class': 'product-name'})
    beer_infos= {}
    
    for row in div_tag:
        dd = row.findAll('dd', attrs={'class': 'small-6 medium-9 columns'})
        beer_infos = {'Nom': name.text.split('-')[0].strip(),
                      'Style': row.find('a').text.strip(),
                      'Contenu': int(row.find('dd', attrs={'class': 'small-6 medium-9 columns js-beer-volume'}).text.strip()[0]),
                      'Degré d’alcool': float(dd[1].text[:-1].replace(",", ".")),
                      'Origine': row.find('dd', attrs={'class': 'small-6 medium-9 columns js-beer-country'}).text.strip(),
                      'Brasseur': dd[2].text.strip()
        }
        
    return beer_infos

extract_beer_infos(URL_PAGE4)

{'Nom': 'Newcastle Brown Ale',
 'Style': 'Bière Brune',
 'Contenu': 2,
 'Degré d’alcool': 4.7,
 'Origine': 'Royaume-Uni',
 'Brasseur': 'Scottish Newcastle'}

In [6]:
# API JSON
response = requests.get(f'https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille')
data = response.json()
data

{'searchTerm': None,
 'totalMatching': 4,
 'pageSize': 48,
 'pageNumber': 1,
 'shouldCurrentUserSeeReservedStock': False,
 'currency': '€',
 'categoryHeader': None,
 'items': [{'title': 'La Cristal IPA du Mont Blanc',
   'description': '',
   'images': [{'image': '/globalassets/la-cristal-ipa-du-mont-blanc.33_1_1.png',
     'badge': {'url': '/globalassets/catalog/beerwulf/beers/awards/wba-france-gold-2018.png',
      'alt': 'WBA France Gold 2018',
      'title': 'WBA France Gold 2018'},
     'name': None}],
   'contentReference': '/fr-fr/p/bieres/la-cristal-ipa-du-mont-blanc.33',
   'redirectedUrl': None,
   'details': 'IPA | 4,7% | 33 cl',
   'worksWithLabel': None,
   'productIcon': None,
   'searchItemType': 0,
   'promotedPages': [],
   'code': 'La-Cristal-IPA-du-Mont-Blanc.33_1',
   'inStock': True,
   'isComingSoon': False,
   'isBottleOrCan': True,
   'inReserveStock': True,
   'style': 'IPA',
   'alcoholPercentage': 4.7,
   'volume': 33.0,
   'displayInformationPrice': {'curren

In [14]:
# partie B-2
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

def extract_beer_list_infos(url):
    
    # Collecter les pages de bières à partir du JSON
    res = requests.get(url)
    data = res.json()
    
    beer_pages = []
    
    # Sequential version (slow):
    beers = []
    
    for element in data['items']:
        link = element['contentReference']
        beer_pages.append(f'https://www.beerwulf.com/{link}')

    for url in beer_pages:
        #infos = extract_beer_infos(url)
        beers.append(extract_beer_infos(url))

    # Facultatif
    # Parallel version (faster):
    # beers = []
    
    return beers

In [8]:
import unittest

class Session4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        self.assertIsInstance(infos, list)
        self.assertGreater(len(infos), 1)
        # Contenu = int
        # Degré d’alcool = float
        for beer in infos:
            self.assertIsInstance(beer['Nom'], str)
            self.assertIsInstance(beer['Style'], str)
            self.assertIsInstance(beer['Contenu'], int)
            self.assertIsInstance(beer['Degré d’alcool'], float)
            self.assertEqual(beer['Origine'], "France")
            self.assertIsInstance(beer['Brasseur'], str)

            
def run_tests():
    test_suite = unittest.makeSuite(Session4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [15]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Session4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Session4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Session4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 5.358s

OK
